In [ ]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import community.community_louvain as community_louvain
import funkcie as fn
fn.setup_directories()

#### NAČÍTANIE DÁT

In [ ]:
G = fn.load_data("facebook_combined.txt")
pos = nx.spring_layout(G, seed=42)
fn.draw_graph_raw(G, pos)

Načítavam dáta z: facebook_combined.txt
-> Uzlov: 4039
-> Hrán: 88234
Počítam layout (môže to chvíľu trvať)...


#### VYHODNOTENIE

In [5]:
# Zozbieranie výsledkov do zoznamu
results = [
    {"Model": "Louvain", "Modularita": round(mod_louvain, 4), "Počet komunít": count_louvain},
    {"Model": "LPA",     "Modularita": round(mod_lpa, 4),     "Počet komunít": count_lpa}
]

# Uloženie a zobrazenie tabuľky
df_results = fn.save_results_table(results)
df_results

,Model,Modularita,Počet komunít
0,Louvain,0.8349,15
1,LPA,0.7368,44


#### EXPERIMENT A: Vlpyv hyperparametra resolution na výsledky modelu Louvain

In [ ]:
# ==========================================
# EXPERIMENT A: Ladenie parametra Resolution
# ==========================================
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import community.community_louvain as community_louvain
import funkcie as fn  # Importujeme tvoje pomocné funkcie

# 1. Definícia rozsahu resolution
resolutions = np.arange(0, 2.1, 0.1)

results_exp_a = []
best_modularity = -1
best_resolution = 1.0
best_partition = None

# --- VÝPOČTY ---
for r in resolutions:
    r = round(r, 1)
    
    try:
        # Výpočet Louvain (tento voláme priamo, lebo ho v funkcie.py nemáme obalený pre rôzne resolution)
        partition = community_louvain.best_partition(G, resolution=r, random_state=42)
        
        # Výpočet metrík - POUŽITIE TVOJEJ FUNKCIE Z funkcie.py
        mod = fn.calculate_modularity(G, partition)
        n_comms = len(set(partition.values()))
        
        results_exp_a.append({
            "Resolution": r,
            "Modularita": mod,
            "Počet komunít": n_comms
        })
        
        if mod > best_modularity:
            best_modularity = mod
            best_resolution = r
            best_partition = partition
            
    except Exception as e:
        pass

df_a = pd.DataFrame(results_exp_a)

# -------------------------------------------------------
# 2. Uloženie GRAFU (bez zobrazenia)
# -------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Resolution Parameter')
ax1.set_ylabel('Modularita', color=color, fontweight='bold')
ax1.plot(df_a['Resolution'], df_a['Modularita'], color=color, marker='o', markersize=4, linewidth=2)
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Počet komunít', color=color, fontweight='bold')
ax2.plot(df_a['Resolution'], df_a['Počet komunít'], color=color, marker='s', markersize=4, linestyle='--', linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)

ax1.set_xticks(np.arange(0, 2.1, 0.1))
plt.title(f"Experiment A: Vplyv Resolution ({resolutions[0]} - {resolutions[-1]})")
plt.savefig("img/experiment_A_resolution_fine.png", dpi=300, bbox_inches='tight')
plt.close()

# -------------------------------------------------------
# 3. Uloženie TABUĽKY výsledkov do PNG
# -------------------------------------------------------
# Príprava dát pre tabuľku (formátovanie na text)
cell_text = []
for index, row in df_a.iterrows():
    cell_text.append([
        f"{row['Resolution']:.1f}",
        f"{row['Modularita']:.4f}",
        f"{int(row['Počet komunít'])}"  # Celé číslo
    ])

# Nastavenie výšky obrázka
num_rows = len(df_a)
fig_height = (num_rows * 0.3) + 1.0 

fig_tbl, ax_tbl = plt.subplots(figsize=(6, fig_height))
ax_tbl.axis('tight')
ax_tbl.axis('off')

# Vytvorenie tabuľky
table = ax_tbl.table(cellText=cell_text,
                     colLabels=["Resolution", "Modularita", "Počet komunít"],
                     cellLoc='center',
                     loc='center')

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

plt.title("Detailné výsledky Experimentu A", y=1.0, pad=20, fontsize=12, fontweight='bold')
plt.savefig("img/experiment_A_tabulka.png", dpi=300, bbox_inches='tight')
plt.close()

# -------------------------------------------------------
# 4. Uloženie VÍŤAZNEJ vizualizácie - POUŽITIE TVOJEJ FUNKCIE
# -------------------------------------------------------
fn.draw_communities(G, best_partition, pos, 
                   f"Najlepší model (Louvain res={best_resolution})", 
                   "img/best_louvain_model.png")